In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# 1. Transformations
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

dataset = datasets.ImageFolder(
    root="/kaggle/input/indian-food-classification/Food Classification/",
    transform=data_transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class_names = dataset.classes

In [2]:
import numpy as np
import torch
import os

In [3]:
data_set = "/kaggle/input/indian-food-classification/Food Classification/"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
print("Classes:", class_names)
print(f"Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")

Classes: ['burger', 'butter_naan', 'chai', 'chapati', 'chole_bhature', 'dal_makhani', 'dhokla', 'fried_rice', 'idli', 'jalebi', 'kaathi_rolls', 'kadai_paneer', 'kulfi', 'masala_dosa', 'momos', 'paani_puri', 'pakode', 'pav_bhaji', 'pizza', 'samosa']
Train size: 5015, Test size: 1254


In [6]:
import torchvision.models as models
import torch.nn as nn

model_resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

for param in model_resnet.parameters():
    param.requires_grad = False

model_resnet.fc = nn.Linear(in_features=model_resnet.fc.in_features, 
                            out_features=len(dataset.classes))

model_resnet = model_resnet.to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 171MB/s] 


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_resnet.parameters(), lr=0.001)

In [8]:
print(len(dataset.classes))

20


In [9]:
def train_step(model, dataloader, optimizer, loss_fn, device):
    model.train()
    total_loss, total_correct = 0, 0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)

        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(y_pred, dim=1)
        total_correct += (preds == y).sum().item()

    avg_loss = total_loss / len(dataloader)
    avg_acc = total_correct / len(dataloader.dataset)
    return avg_loss, avg_acc

In [10]:
def test_step(model, dataloader, loss_fn, device):
    model.eval()
    total_loss, total_correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            y_pred = model(X)
            loss = loss_fn(y_pred, y)

            total_loss += loss.item()
            preds = torch.argmax(y_pred, dim=1)
            total_correct += (preds == y).sum().item()

    avg_loss = total_loss / len(dataloader)
    avg_acc = total_correct / len(dataloader.dataset)
    return avg_loss, avg_acc

In [11]:
epochs = 20
for epoch in range(epochs):
    train_loss, train_acc = train_step(model_resnet, train_dataloader, optimizer, loss_fn, device)
    test_loss, test_acc = test_step(model_resnet, test_dataloader, loss_fn, device)

    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

Epoch [1/20] Train Loss: 2.6795, Train Acc: 0.2146 | Test Loss: 2.2904, Test Acc: 0.3429
Epoch [2/20] Train Loss: 2.0588, Train Acc: 0.3938 | Test Loss: 2.0710, Test Acc: 0.4011
Epoch [3/20] Train Loss: 1.8429, Train Acc: 0.4632 | Test Loss: 2.0251, Test Acc: 0.4211
Epoch [4/20] Train Loss: 1.7335, Train Acc: 0.4983 | Test Loss: 1.9666, Test Acc: 0.4314
Epoch [5/20] Train Loss: 1.6800, Train Acc: 0.5115 | Test Loss: 1.9127, Test Acc: 0.4514
Epoch [6/20] Train Loss: 1.5996, Train Acc: 0.5348 | Test Loss: 1.9302, Test Acc: 0.4354
Epoch [7/20] Train Loss: 1.5654, Train Acc: 0.5262 | Test Loss: 1.9845, Test Acc: 0.4226
Epoch [8/20] Train Loss: 1.5309, Train Acc: 0.5422 | Test Loss: 1.9665, Test Acc: 0.4330
Epoch [9/20] Train Loss: 1.5095, Train Acc: 0.5498 | Test Loss: 1.9486, Test Acc: 0.4394
Epoch [10/20] Train Loss: 1.4848, Train Acc: 0.5531 | Test Loss: 1.9735, Test Acc: 0.4258
Epoch [11/20] Train Loss: 1.4938, Train Acc: 0.5500 | Test Loss: 1.9712, Test Acc: 0.4386
Epoch [12/20] Train